## Настраиваем окружение

In [34]:
import folium
import pandas as pd
import numpy as np
import webbrowser

In [35]:
peregon = pd.read_excel("D:\MyFolder\\2.Work\DataWagon\PEREGON_HACKATON.xlsx")
station_coord = pd.read_excel("D:\MyFolder\\2.Work\DataWagon\STATION_COORDS_HACKATON.xlsx")
disl = pd.read_excel("D:\MyFolder\\2.Work\DataWagon\disl_hackaton.xlsx")

In [39]:
station_coord_cleaned = station_coord.dropna()


## Проверка идеи

In [46]:
# Найти строки с NaN значениями
nan_rows = disl[disl.isnull().any(axis=1)]

# Вывести найденные строки
print(nan_rows)


        WAGNUM            OPERDATE  ST_ID_DISL  ST_ID_DEST TRAIN_INDEX
282814    6167 2023-06-12 23:50:00        9465        1227         NaN
282820    6167 2023-06-09 14:03:00       16947        1227         NaN
283042    6167 2023-06-11 18:52:00        9435        1227         NaN
283143    6167 2023-06-10 19:52:00        9448        1227         NaN
283650    6273 2023-06-12 23:50:00        9465        1227         NaN
283653    6273 2023-06-09 14:03:00       16947        1227         NaN
283980    6273 2023-06-11 18:52:00        9435        1227         NaN
284097    6273 2023-06-10 19:52:00        9448        1227         NaN
287940    7173 2023-06-12 23:50:00        9465        1227         NaN
287945    7173 2023-06-09 14:03:00       16947        1227         NaN
288283    7173 2023-06-11 18:52:00        9435        1227         NaN
288440    7173 2023-06-10 19:52:00        9448        1227         NaN
332566    9765 2023-07-22 14:00:00       20695        1561         NaN
332582

## Отрисовка карты

In [120]:
# Создаем карту
m = folium.Map(tiles="Cartodb dark_matter",location=[station_coord_cleaned['LATITUDE'].mean(), station_coord_cleaned['LONGITUDE'].mean()], zoom_start=5)
# Добавляем статичные маркеры (CircleMarker) для каждого объекта
for index, row in station_coord_cleaned.iterrows():
    folium.CircleMarker(location=[row['LATITUDE'], row['LONGITUDE']],
                        radius=1,  # радиус круга (можно настроить)
                        color='white',  # цвет круга
                        fill=True,
                        fill_color='white',  # цвет заливки
                        fill_opacity=0.7,  # прозрачность заливки
                        popup=row['ST_ID']).add_to(m)
# Добавляем линии между соответствующими вершинами

for index, row in peregon.iterrows():
    start_station = station_coord_cleaned.loc[station_coord_cleaned['ST_ID'] == row['START_CODE']]
    end_station = station_coord_cleaned.loc[station_coord_cleaned['ST_ID'] == row['END_CODE']]
    
    if not start_station.empty and not end_station.empty:
        start_coords = [start_station['LATITUDE'].values[0], start_station['LONGITUDE'].values[0]]
        end_coords = [end_station['LATITUDE'].values[0], end_station['LONGITUDE'].values[0]]
        
        folium.PolyLine(locations=[start_coords, end_coords], color='white', weight=2, opacity=0.7).add_to(m)

# Сохраняем карту в HTML-файл
m.save('base_map.html')

## Эмулируем поступление данных

In [121]:
# Сортировка данных по столбцу OPERDATE
sorted_df = disl.sort_values(by='OPERDATE')

# Группировка данных по столбцу OPERDATE
grouped_df = sorted_df.groupby('OPERDATE')


In [122]:
first_data_input = sorted_df[0:100].reset_index()

In [49]:
first_data_input

,index,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX
0,340706,6516,2023-06-01 00:00:00,1268,1655,1268-677-1663
1,339038,5100,2023-06-01 00:00:00,1268,1655,1268-677-1663
2,340705,6516,2023-06-01 00:00:01,1268,1655,1268-677-1663
3,339037,5100,2023-06-01 00:00:01,1268,1655,1268-677-1663
4,88525,6394,2023-06-01 00:01:00,4576,633,20715-079-20713
...,...,...,...,...,...,...
95,984338,3041,2023-06-01 00:24:00,13913,2331,16069-096-2331
96,923814,8966,2023-06-01 00:24:00,13913,2331,16069-096-2331
97,88527,6394,2023-06-01 00:24:00,4571,633,20715-079-20713
98,955631,8482,2023-06-01 00:26:00,1839,2331,16069-088-2331


In [123]:
# Преобразуем столбцы с датами в формат datetime
first_data_input['OPERDATE'] = pd.to_datetime(first_data_input['OPERDATE'])

# 1. Взять самую новую информацию по каждому вагону
latest_data = first_data_input.sort_values(by='OPERDATE', ascending=False).groupby('WAGNUM').head(1)


## Добавляем на карту поезда

In [124]:

for index, row in latest_data.iterrows():
    # Добавляем маркер текущего расположения вагона
    current_location = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DISL']]
    if not current_location.empty:
        folium.Marker(location=[current_location['LATITUDE'].values[0],
                                current_location['LONGITUDE'].values[0]],
                    popup=f"<div>Train: {row['TRAIN_INDEX']},\n Destination: {row['ST_ID_DEST']},\n<\div>").add_to(m)

# # 3. При нажатии на вагон отметить точку назначения текущего вагона
# for index, row in latest_data.iterrows():
#     # Добавляем всплывающий маркер для точки назначения
#     destination_location = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DEST']]
#     if not destination_location.empty:
#         folium.Marker(location=[destination_location['LATITUDE'].values[0], destination_location['LONGITUDE'].values[0]],
#                     popup=f"Destination: {row['ST_ID_DEST']}").add_to(m)

# Сохраняем карту в HTML-файл
m.save('train_map.html')
webbrowser.open_new_tab('train_map.html')


KeyboardInterrupt: 

In [145]:
for index, row in latest_data.iterrows():
    # Добавляем маркер текущего расположения вагона
    current_location = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DISL']]
    if not current_location.empty:
        popup_html=fr"""<div >
                        Номер поезда: {row['TRAIN_INDEX']}<br> 
                        Дата обновления: {row["OPERDATE"]}<br>
                        ID станции прибытия: {row['ST_ID_DEST']}<br>
                        ID текущей станции: {row['ST_ID_DISL']}<br>
                        Номер вагона: {row["WAGNUM"]}<br>
                        <\div>"""
        folium.Marker(location=[current_location['LATITUDE'].values[0],
                                current_location['LONGITUDE'].values[0]],
                    popup=folium.Popup(popup_html, min_width=500),
                    icon=folium.Icon(color='orange', icon='leaf', prefix='fa')).add_to(m)


# Сохраняем карту в HTML-файл
m.save('train_map.html')
webbrowser.open_new_tab('train_map.html')

True

In [ ]:
for index, row in latest_data.iterrows():
    # Добавляем маркер текущего расположения вагона
    current_location = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DISL']]
    if not current_location.empty:
        folium.Marker(location=[current_location['LATITUDE'].values[0],
                                current_location['LONGITUDE'].values[0]],
                    #popup=f"Train: {row['TRAIN_INDEX']},\n Destination: {row['ST_ID_DEST']}").add_to(m)
                    popup=f"""<div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; box-shadow: 2px 2px 5px #888888;">
                                    <p>Номер поезда: {row['TRAIN_INDEX']}<\p>
                                    <p>Дата обновления: {row["OPERDATE"]}<\p>
                                    <p>ID текущей станции: {row["ST_ID_DISL"]}<\p>
                                    <p>ID станции прибытия: {row["'ST_ID_DEST'"]}<\p>
                            </div>""".add_to(m))

# 3. При нажатии на вагон отметить точку назначения текущего вагона
# for _ , row in latest_data.iterrows():
#     # Добавляем всплывающий маркер для точки назначения
#     destination_location = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DEST']]
#     if not destination_location.empty:
#         # folium.Marker(location=[destination_location['LATITUDE'].values[0], destination_location['LONGITUDE'].values[0]],
#         #             popup=f"Destination: {row['ST_ID_DEST']}").add_to(m)
#         folium.Marker(location=[destination_location['LATITUDE'].values[0], destination_location['LONGITUDE'].values[0]],
#                     popup=f"""<div style="background-color: #f8f9fa; padding: 10px; border-radius: 5px; box-shadow: 2px 2px 5px #888888;">
#         <p>Номер поезда: {row['TRAIN_INDEX']}<\p>
#         <p>Дата обновления: {row["OPERDATE"]}<\p>
#         <p>ID текущей станции: {row["ST_ID_DISL"]}<\p>
#         <p>ID станции прибытия: {row["'ST_ID_DEST'"]}<\p>
#     </div>
# """.add_to(m))

# Сохраняем карту в HTML-файл
m.save('train_map.html')
webbrowser.open_new_tab('train_map.html')


In [94]:
latest_data

,index,WAGNUM,OPERDATE,ST_ID_DISL,ST_ID_DEST,TRAIN_INDEX
99,927468,8837,2023-06-01 00:26:00,1839,2331,16069-088-2331
98,955631,8482,2023-06-01 00:26:00,1839,2331,16069-088-2331
92,910813,6444,2023-06-01 00:24:00,13913,2331,16069-096-2331
86,938027,3090,2023-06-01 00:24:00,13913,2331,16069-096-2331
87,995705,8625,2023-06-01 00:24:00,13913,2331,16069-096-2331
...,...,...,...,...,...,...
24,199914,3342,2023-06-01 00:10:00,710,1221,686-880-1176
27,101488,3559,2023-06-01 00:10:00,17506,1138,20792-574-20713
17,9041,576,2023-06-01 00:08:00,1043,113,95-088-1043
2,340705,6516,2023-06-01 00:00:01,1268,1655,1268-677-1663


In [81]:
# webbrowser.open_new_tab('train_map.html')

True

## Пока не нужный код

In [51]:
# Предположим, что first_data_input - это ваш исходный датафрейм
# Сортируем по дате операции в обратном порядке и выбираем первую запись для каждого вагона
latest_data = first_data_input.sort_values(by='OPERDATE', ascending=False).groupby('WAGNUM').first().reset_index()


In [52]:


# Добавляем линии между соответствующими вершинами
for index, row in peregon.iterrows():
    start_station = station_coord.loc[station_coord['ST_ID'] == row['START_CODE']]
    end_station = station_coord.loc[station_coord['ST_ID'] == row['END_CODE']]
    
    if not start_station.empty and not end_station.empty:
        start_coords = [start_station['LATITUDE'].values[0], start_station['LONGITUDE'].values[0]]
        end_coords = [end_station['LATITUDE'].values[0], end_station['LONGITUDE'].values[0]]
        
        folium.PolyLine(locations=[start_coords, end_coords], color='blue', weight=2, opacity=0.7).add_to(m)

# Добавляем маркеры для текущего расположения вагона
for index, row in latest_data.iterrows():
    station = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DISL']]
    if not station.empty:
        coords = [station['LATITUDE'].values[0], station['LONGITUDE'].values[0]]
        folium.Marker(location=coords, popup=f"Wagon: {row['WAGNUM']}").add_to(m)

# Сохраняем карту в HTML-файл
m.save('wagon_map.html')


In [53]:
# Добавляем маркеры для точек назначения вагонов
for index, row in latest_data.iterrows():
    destination_station = station_coord.loc[station_coord['ST_ID'] == row['ST_ID_DEST']]
    if not destination_station.empty:
        destination_coords = [destination_station['LATITUDE'].values[0], destination_station['LONGITUDE'].values[0]]
        
        # Создаем маркер с точкой назначения
        destination_marker = folium.Marker(location=destination_coords, popup=f"Destination for Wagon {row['WAGNUM']}")
        
        # Создаем всплывающее окно с информацией по вагону при нажатии
        destination_marker.add_child(folium.Popup(f"Wagon: {row['WAGNUM']}<br>Train Index: {row['TRAIN_INDEX']}"))
        
        # Добавляем маркер точки назначения на карту
        destination_marker.add_to(m)

# Сохраняем карту в HTML-файл
m.save('wagon_map_with_destination.html')
